In [1]:
import numpy as np
import pandas as pd
import tifffile
import napari
from scipy import ndimage as ndi
from skimage.filters import threshold_otsu,gaussian
from skimage.morphology import ball, binary_dilation, binary_opening
from skimage import exposure
from skimage.exposure import rescale_intensity
from skimage.measure import label, regionprops
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from sklearn.metrics import pairwise_distances
import json
import zarr
from ast import literal_eval
pd.options.mode.chained_assignment = None

### Helper functions

In [2]:
def erase_small_in_z(labels, min_size):
    rps = regionprops(labels)
    for r in rps:
        len_in_z = r.bbox[3]-r.bbox[0]
        if len_in_z<min_size:
            labels[labels==r.label]=0
    return labels

In [3]:
def erase_small(labels, min_size):
    unique, counts = np.unique(labels, return_counts=True)
    for i in range(len(unique)):
        if counts[i]<min_size:
            labels[labels==unique[i]]=0
    return labels

In [4]:
def remove_backg(img, sig1=1,sig2=15):
    img_sig =ndi.gaussian_filter(img, (sig1, sig1,sig1))
    img_bcg = ndi.gaussian_filter(img, (sig2,sig2,sig2))
    cleaned = img_sig - img_bcg
    cleaned[cleaned<0]=0
    return cleaned

In [5]:
def create_immuno_mask(img, bcg,perc=99.5,sig1=2,sig2=10):
    img = img.astype('float32')
    mi = 101
    ma = np.percentile(img, perc)
    img_rescale = rescale_intensity(img, in_range=(mi, ma))
    img_clean =remove_backg(img_rescale,sig1,sig2)
    
    threshold =  threshold_otsu(img_clean)
    
    mask = img_clean > threshold
    labels = label(mask)
    rp = regionprops(labels, intensity_image = bcg)
    max_intensity = []
    labels_list = []
    
    for r in rp:
        labels_list.append(r.label)
        max_intensity.append(np.amax(r.intensity_image))
    data = list(zip(labels_list, max_intensity))
    df = pd.DataFrame(data = data ,columns = ["label", "max_intensity"])
    intensity_threshold = threshold_otsu(df["max_intensity"].values)
    print(intensity_threshold)
    df['is_synapse'] = np.where(df['max_intensity'] > intensity_threshold, 'Yes', 'No')
    labels_real = df[df["is_synapse"]=='Yes']
    real_labels = list(labels_real["label"])
    img_real_labels = np.isin(labels,real_labels)
    img_real_labels  = labels*img_real_labels
    mask_filtered = img_real_labels>0
    return mask_filtered

In [6]:
def edistance (loc1,loc2):
    return np.sqrt((loc1[0]-loc2[0])**2 + (loc1[1]-loc2[1])**2 + (loc1[2]-loc2[2])**2)

In [7]:
def cost(location1, location2, matching_threshold):
    max_cost = 2 * matching_threshold

    # First check of the nodes are part of the same segment
    pre_dist = edistance(location1, location2)
    if pre_dist > matching_threshold:
        return max_cost
    dist = pre_dist
    return dist

In [32]:
def cost_matrix(predicted, immuno, matching_threshold):
    print("Computing matching costs...")

    size = max(len(predicted), len(immuno))
    costs = np.zeros((size, size), dtype=np.float32)
    costs[:] = 2 * matching_threshold
    num_potential_matches = 0
    for i in range(len(predicted)):
        for j in range(len(immuno)):
            c = cost(predicted[i], immuno[j], matching_threshold)
            costs[i, j] = c
            if c <= matching_threshold:
                num_potential_matches += 1

    print(str(num_potential_matches) + " potential matches found")

    return costs

In [33]:
def compute_molecule_detection_quality(locs_gt, locs_detected, matching_threshold):
    costs = cost_matrix(locs_detected, locs_gt, matching_threshold)
    matches = linear_sum_assignment(costs - np.amax(costs) - 1)
    matches = zip(matches[0], matches[1])  # scipy returns matches as numpy arrays

    filtered_matches = [(i, j) for (i, j) in matches if costs[i][j] <= matching_threshold]
    fp = len(locs_detected) - len(filtered_matches)

    # unmatched in gt = FN
    fn = len(locs_gt) - len(filtered_matches)
    
        # all ground truth elements - FN = TP
    tp = len(locs_gt) - fn
    
    precision = float(tp) / (tp + fp) if (tp + fp) > 0 else 0
    recall = float(tp) / (tp + fn) if (tp + fn) > 0 else 0
    if (precision + recall) > 0:
        fscore = 2.0 * precision * recall / (precision + recall)
    else:
        fscore = 0.0
    print("TP:" + str(tp))
    print("FP:" + str(fp))
    print("FN:" + str(fn))
    print('Precision:' + str(precision))
    print('Recall:' + str(recall))
    print('F1 score:' + str(fscore))
    return precision,recall,fscore,filtered_matches

In [34]:
def generate_matching_map(presyn_coords,postsyn_coords,matching_threshold=30):
    locs_pre_synapse = np.stack(presyn_coords)
    locs_post_synapse = np.stack(postsyn_coords)
    costs = cost_matrix(locs_post_synapse, locs_pre_synapse,matching_threshold)

    matches = linear_sum_assignment(costs - np.amax(costs) - 1)
    matches = zip(matches[0], matches[1])  # scipy returns matches as numpy arrays

    filtered_matches = [(i, j) for (i, j) in matches if costs[i][j] <= matching_threshold]
    matches_map = np.zeros(costs.shape)

    for i in range(costs.shape[0]):
        post_pre_connection_id = np.argmin(costs[i,:])
        if np.amin(costs[i,:])<=matching_threshold:
            matches_map[i,post_pre_connection_id]=1

    for i in range(costs.shape[0]):
        pre_post_connection_id = np.argmin(costs[:,i])
        if (np.amin(costs[:,i])<=matching_threshold) and (sum(matches_map[:,i])==0):
            matches_map[pre_post_connection_id ,i]=1
    return matches_map,filtered_matches

In [35]:
def match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon,filtered_matches_shank2):

    inv_filtered_matches_shank2 = dict((v, k) for k, v in dict(filtered_matches_shank2).items())
    inv_filtered_matches_bassoon = dict((v, k) for k, v in dict(filtered_matches_bassoon).items())

    full_synapses_gt_and_detected = full_synapses_gt
    full_synapses_gt_and_detected['post_id_detected_matched'] = -1
    full_synapses_gt_and_detected['pre_id_detected_matched'] = -1

    for i in range(len(full_synapses_gt_and_detected)):
        post_id = full_synapses_gt_and_detected['post_id'].loc[i]
        ind = post_synapses_gt[post_synapses_gt['id']==post_id].index[0]
        if ind in dict(inv_filtered_matches_shank2):
            full_synapses_gt_and_detected['post_id_detected_matched'].loc[i] = dict(inv_filtered_matches_shank2)[ind]

    for i in range(len(full_synapses_gt_and_detected)):
        pre_id = full_synapses_gt_and_detected['pre_id'].loc[i]
        ind = pre_synapses_gt[pre_synapses_gt['id']==pre_id].index[0]
        if ind in dict(inv_filtered_matches_bassoon):
            full_synapses_gt_and_detected['pre_id_detected_matched'].loc[i] = dict(inv_filtered_matches_bassoon)[ind]

    return full_synapses_gt_and_detected

In [36]:
def compute_full_synapse_detection_quality(full_synapses_gt_and_detected,matches_map):
#add column that stores information about the detection
    full_synapses_gt_and_detected['Detection is correct'] = 'No'

    #check if the deected pre-and-post are connected inn the detection as well
    for i in range(len(full_synapses_gt_and_detected)):
        post_id = full_synapses_gt_and_detected['post_id_detected_matched'].loc[i]
        pre_id = full_synapses_gt_and_detected['pre_id_detected_matched'].loc[i]
        if matches_map[post_id,pre_id]>0:
            full_synapses_gt_and_detected['Detection is correct'].loc[i]='Yes'
            matches_map[post_id,pre_id]=2

    un, counts = np.unique(matches_map, return_counts = True)
    fp = counts[1]
    print('FP full-synapse:' + str(fp))
    fp_ids = np.where(matches_map==1)
    fp_ids_post = fp_ids[0]
    fp_ids_pre = fp_ids[1]

    tps = full_synapses_gt_and_detected[full_synapses_gt_and_detected["Detection is correct"]=='Yes']
    tp = len(tps)
    print('TP full-synapse:' + str(tp))

    fns = full_synapses_gt_and_detected[full_synapses_gt_and_detected["Detection is correct"]=='No']
    fn = len(fns)
    print('FN full-synapse:' + str(fn))
   
    precision = float(tp) / (tp + fp) if (tp + fp) > 0 else 0
    recall = float(tp) / (tp + fn) if (tp + fn) > 0 else 0
    if (precision + recall) > 0:
        fscore = 2.0 * precision * recall / (precision + recall)
    else:
        fscore = 0.0

    print('Precision:' + str(precision))
    print('Recall:' + str(recall))
    print('F1 score:' + str(fscore))
    return precision,recall,fscore

### 1. Load imaging data

In [13]:
img_path = '../../Data/LICONN_test_dataset.tif'
gt_image = tifffile.imread(img_path)

In [14]:
bassoon = gt_image[:,1,:,:]
shank2 = gt_image[:,0,:,:]
liconn = gt_image[:,2,:,:]

In [15]:
viewer = napari.Viewer()
viewer.add_image(liconn,colormap='gray')
viewer.add_image(bassoon,colormap = 'cyan', blending = 'additive')
viewer.add_image(shank2,colormap = 'magenta', blending = 'additive')

<Image layer 'shank2' at 0x1ecf1574ca0>

### 2. Load GT annotations

In [16]:
# Opening JSON file
f = open('../../Data/LICONN_test_dataset_full_synapses.json')
data = json.load(f)

In [17]:
pre_synapses_gt = pd.DataFrame(columns = ['id', 'location'])
post_synapses_gt = pd.DataFrame(columns = ['id', 'location'])
full_synapses_gt = pd.DataFrame(columns = ['post_id', 'post_coordinate', 'pre_id', 'pre_coordinate'])

In [18]:
for i in range(len(data['partners'])):
    
    pre_id = int(data['partners'][i]['pre-synapse'])
    pre_location = data['ids'][str(pre_id)]['location']
    data_pre = [[pre_id, pre_location[::-1]]]
    pre_synapses_gt = pd.concat([pre_synapses_gt,pd.DataFrame(columns = ['id', 'location'], data = data_pre)],ignore_index=True)
    
    post_id = int(data['partners'][i]['post-synapse'])
    post_location = data['ids'][str(post_id)]['location']
    data_post = [[post_id, post_location[::-1]]]
    post_synapses_gt = pd.concat([post_synapses_gt,pd.DataFrame(columns = ['id', 'location'], data = data_post)],ignore_index=True)
    
    full_data = [[post_id, post_location[::-1], pre_id, pre_location[::-1]]]
    full_synapses_gt =pd.concat([full_synapses_gt,pd.DataFrame(columns = ['post_id', 'post_coordinate', 'pre_id', 'pre_coordinate'], data = full_data)], ignore_index=True)

In [19]:
pre_synapses_gt = pre_synapses_gt.drop_duplicates(subset=["id"], keep='first')
post_synapses_gt = post_synapses_gt.drop_duplicates(subset=["id"], keep='first')

In [20]:
f = open('../../Data/LICONN_test_dataset_single_pre.json')
data = json.load(f)

In [21]:
pre_synapses_gt_single = pd.DataFrame(columns = ['id', 'location'])
post_synapses_gt_single = pd.DataFrame(columns = ['id', 'location'])

In [22]:
for i in list(data['ids']):
    pre_id = int(i)
    pre_location = data['ids'][str(pre_id)]['location']
    data_pre = [[pre_id, pre_location[::-1]]]
    pre_synapses_gt_single = pd.concat([pre_synapses_gt_single,pd.DataFrame(columns = ['id', 'location'], data = data_pre)],ignore_index=True)

In [23]:
f = open('../../Data/LICONN_test_dataset_single_post.json')
data = json.load(f)

In [24]:
for i in list(data['ids']):
    post_id = int(i)
    post_location = data['ids'][str(post_id)]['location']
    data_post = [[post_id, post_location[::-1]]]
    post_synapses_gt_single = pd.concat([post_synapses_gt_single,pd.DataFrame(columns = ['id', 'location'], data = data_post)],ignore_index=True)

In [25]:
pre_synapses_gt = pd.concat([pre_synapses_gt,pre_synapses_gt_single],ignore_index=True)
post_synapses_gt = pd.concat([post_synapses_gt,post_synapses_gt_single],ignore_index=True)

In [26]:
locs_bassoon_gt = np.stack(pre_synapses_gt['location'].values)
locs_shank2_gt = np.stack(post_synapses_gt['location'].values)

### 3. Detect synases from immuo signal

##### 3.1 bassoon (pre-synapse) detection

In [27]:
bassoon_immuno_mask = create_immuno_mask(bassoon,liconn,99.5,5,11)

373


In [28]:
labels_basson_immuno = label(bassoon_immuno_mask)
labels_basson_immuno = erase_small(labels_basson_immuno, 60)
rp = regionprops(labels_basson_immuno, intensity_image = liconn)

# save not z-filtered results for the final post-processing step
presyn_coords_immuno_nozclean = []
for r in rp:
    presyn_coords_immuno_nozclean.append(np.array(r.centroid_weighted))
    
labels_basson_immuno = erase_small_in_z(labels_basson_immuno, 13)
rp = regionprops(labels_basson_immuno, intensity_image = liconn)
presyn_coords_immuno = []
for r in rp:
    presyn_coords_immuno.append(np.array(r.centroid_weighted))

##### 3.2 shank2 (post-synapse) detection

In [29]:
shank2_immuno_mask = create_immuno_mask(shank2,liconn,99,4,11)

387


In [30]:
labels_shank2_immuno = label(shank2_immuno_mask)
labels_shank2_immuno = erase_small(labels_shank2_immuno, 60)
rp = regionprops(labels_shank2_immuno, intensity_image = liconn)

# save not z-filtered results for the final post-processing step
postsyn_coords_immuno_nozclean = []
for r in rp:
    postsyn_coords_immuno_nozclean.append(np.array(r.centroid_weighted))
    
labels_shank2_immuno = erase_small_in_z(labels_shank2_immuno, 13)
rp = regionprops(labels_shank2_immuno, intensity_image = liconn)
postsyn_coords_immuno = []
for r in rp:
    postsyn_coords_immuno.append(np.array(r.centroid_weighted))

##### 3.3 match pre and post synapses to detect full synapses (generate matches map)

In [31]:
matches_map_initial,filtered_matches_immuno_detecion = generate_matching_map(presyn_coords_immuno,postsyn_coords_immuno,matching_threshold=30)

Computing matching costs...
221 potential matches found


### 4. Validation

##### 4.1 Single bassoon and shank2 detection validation

In [37]:
matching_threshold = 30

In [38]:
precision,recall,fscore,filtered_matches_bassoon = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_immuno), matching_threshold)

Computing matching costs...
253 potential matches found
TP:232
FP:20
FN:29
Precision:0.9206349206349206
Recall:0.8888888888888888
F1 score:0.9044834307992203


In [39]:
precision,recall,fscore,filtered_matches_shank2 = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_immuno), matching_threshold)

Computing matching costs...
242 potential matches found
TP:214
FP:2
FN:33
Precision:0.9907407407407407
Recall:0.8663967611336032
F1 score:0.9244060475161986


##### 4.2 Full synapse detection quality

In [40]:
#match detected bassoon and shank2 with full gt synapses
full_synapses_gt_and_immuno = match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon,filtered_matches_shank2)

In [41]:
#compute F1 score of full synapse detection
fprecision,frecall,ffscore = compute_full_synapse_detection_quality(full_synapses_gt_and_immuno,matches_map_initial)

FP full-synapse:13
TP full-synapse:201
FN full-synapse:41
Precision:0.9392523364485982
Recall:0.8305785123966942
F1 score:0.8815789473684211


### Post-processing

##### Second look proofreading step

In [42]:
def get_new_detections(bbox,gt_image,labels_bassoon,labels_shank2,current_presyn_coordinate, matching_threshold):
    z0,y0,x0,z1,y1,x1 = bbox
    bbassoon = gt_image[z0:z1,1,y0:y1,x0:x1]
    bshank2 = gt_image[z0:z1,0,y0:y1,x0:x1]
    bliconn = gt_image[z0:z1,2,y0:y1,x0:x1]
    mi = np.percentile(bliconn, 1)
    ma = np.percentile(bliconn, 99.95)
    img_rescale = rescale_intensity(bliconn, in_range=(mi, ma))
    cleaned_pan = remove_backg(img_rescale.astype('float32'), sig1=2,sig2=10)
    mi = np.percentile(cleaned_pan, 95)
    ma = np.percentile(cleaned_pan, 100)
    threshold = 0.4*(ma-mi) + mi
    pan_mask = cleaned_pan>threshold
    mask_existing_labels = labels_shank2[z0:z1,y0:y1,x0:x1] + labels_bassoon[z0:z1,y0:y1,x0:x1]
    mask_existing_labels = mask_existing_labels>0
    potential_shank2_mask = pan_mask*~mask_existing_labels
    potential_shank2_mask = binary_opening(potential_shank2_mask, ball(2))
    labels_potential_shank2 = label(potential_shank2_mask)
    labels_potential_shank2 = erase_small(labels_potential_shank2, 20)
    rp = regionprops(labels_potential_shank2, intensity_image = bliconn)
    new_postsyn_detections = []
    for r in rp:
        new_postsyn_detections.append(np.array(r.centroid_weighted))
    final_new_shank2_list = []
    for i in range(len(new_postsyn_detections)):
        shank2_pos = new_postsyn_detections[i] + [z0,y0,x0]
        bassoon_pos = current_presyn_coordinate
        if edistance (shank2_pos,bassoon_pos )<matching_threshold:
            final_new_shank2_list.append(shank2_pos)  
    return final_new_shank2_list

In [43]:
doublecheck_detection_bassoon_list = []
#find all flying bassoons without a pair
for i in range(len(matches_map_initial)):
    if sum(matches_map_initial[:,i])==0: #and i not in bassoon_detected_ids_to_remove:
        doublecheck_detection_bassoon_list.append(i)

In [44]:
doublecheck_detection_shank2_list = []
#find all flying bassoons without a pair
for i in range(len(matches_map_initial)):
    if sum(matches_map_initial[i,:])==0: #and i not in shank2_detected_ids_to_remove:
        doublecheck_detection_shank2_list.append(i)

In [45]:
bboxes = []
search_dist = 50
i=0
all_new_shank2 = []
for ind in doublecheck_detection_bassoon_list:
    coordinate = presyn_coords_immuno[ind].astype('uint16')
    z0 = max(0,coordinate[0]-search_dist)
    z1 = min(coordinate[0] + search_dist, bassoon.shape[0])
    z_mid = ((z1-z0)/2).astype('uint16')
    y0 = max(0,coordinate[1]-search_dist)
    y1 = min(coordinate[1] + search_dist, bassoon.shape[1])
    x0 = max(0,coordinate[2]-search_dist)
    x1 = min(coordinate[2] + search_dist, bassoon.shape[2])
    bboxes.append([z0,y0,x0,z1,y1,x1])  
    final_new_shank2_list = get_new_detections([z0,y0,x0,z1,y1,x1],gt_image,labels_basson_immuno,labels_shank2_immuno,coordinate, matching_threshold)
    if len(final_new_shank2_list)!=0:
        for j in range(len(final_new_shank2_list)):
            all_new_shank2.append(final_new_shank2_list[j])

In [46]:
postsyn_coords_immuno_after_doublecheck = postsyn_coords_immuno+all_new_shank2

In [47]:
precision,recall,fscore,filtered_matches_shank2 = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_immuno_after_doublecheck), matching_threshold)

Computing matching costs...
255 potential matches found
TP:223
FP:7
FN:24
Precision:0.9695652173913043
Recall:0.902834008097166
F1 score:0.9350104821802935


In [48]:
bboxes = []
search_dist = 50
all_new_bassoons = []
for ind in doublecheck_detection_shank2_list:
    if ind < len(postsyn_coords_immuno):
        coordinate = postsyn_coords_immuno[ind].astype('uint16')
        z0 = max(0,coordinate[0]-search_dist)
        z1 = min(coordinate[0] + search_dist, bassoon.shape[0])
        z_mid = ((z1-z0)/2).astype('uint16')
        y0 = max(0,coordinate[1]-search_dist)
        y1 = min(coordinate[1] + search_dist, bassoon.shape[1])
        x0 = max(0,coordinate[2]-search_dist)
        x1 = min(coordinate[2] + search_dist, bassoon.shape[2])
        bboxes.append([z0,y0,x0,z1,y1,x1])  
        final_new_bassoon_list = get_new_detections([z0,y0,x0,z1,y1,x1],gt_image,labels_basson_immuno,labels_shank2_immuno,coordinate, matching_threshold)
        if len(final_new_bassoon_list)!=0:
            for j in range(len(final_new_bassoon_list)):
                all_new_bassoons.append(final_new_bassoon_list[j])
    #print(final_new_shank2_list)
    #print('---')

In [49]:
presyn_coords_immuno_after_doublecheck = presyn_coords_immuno+all_new_bassoons

In [50]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_immuno_after_doublecheck), matching_threshold)

Computing matching costs...
254 potential matches found
TP:233
FP:20
FN:28
Precision:0.9209486166007905
Recall:0.89272030651341
F1 score:0.9066147859922179


In [51]:
matches_map_doublecheck,  filtered_matches_immuno_detecion = generate_matching_map(presyn_coords_immuno_after_doublecheck,postsyn_coords_immuno_after_doublecheck,matching_threshold=30)

Computing matching costs...
237 potential matches found


In [52]:
#match detected bassoon and shank2 with full gt synapses
full_synapses_gt_and_immuno_after_doublecheck = match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon,filtered_matches_shank2)

In [53]:
#compute F1 score of full synapse detection
fprecision,frecall,ffscore = compute_full_synapse_detection_quality(full_synapses_gt_and_immuno_after_doublecheck,matches_map_doublecheck)

FP full-synapse:18
TP full-synapse:211
FN full-synapse:31
Precision:0.9213973799126638
Recall:0.871900826446281
F1 score:0.8959660297239915


##### Border cleaning step

In [54]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon_nozfilter = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_immuno_nozclean), matching_threshold)

Computing matching costs...
272 potential matches found
TP:244
FP:31
FN:17
Precision:0.8872727272727273
Recall:0.9348659003831418
F1 score:0.9104477611940298


In [55]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon_init = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_immuno_after_doublecheck), matching_threshold)

Computing matching costs...
254 potential matches found
TP:233
FP:20
FN:28
Precision:0.9209486166007905
Recall:0.89272030651341
F1 score:0.9066147859922179


In [56]:
tp_ids_init = list(zip(*filtered_matches_bassoon_init))[1]
tp_ids_nozfilter = list(zip(*filtered_matches_bassoon_nozfilter))[1]
s = set(tp_ids_init)
potential_tps = [x for x in tp_ids_nozfilter if x not in s]

In [57]:
inv_filtered_matches_bassoon_nozfilter = dict((v, k) for k, v in dict(filtered_matches_bassoon_nozfilter).items())

In [58]:
tps_to_keep = []
new_presyn_immuno_detections = []
new_tps=0
for i in potential_tps:
    if locs_bassoon_gt[i][0]<=10 or  locs_bassoon_gt[i][0]>=190:
        tps_to_keep.append(i)
        new_tps+=1
        new_presyn_immuno_detections.append(presyn_coords_immuno_nozclean[inv_filtered_matches_bassoon_nozfilter[i]])
        #here we append the detections matched with these GTs backt o the presyn_coords_immuno list

In [59]:
presyn_coords_immuno_after_doublecheck_borders = new_presyn_immuno_detections + presyn_coords_immuno_after_doublecheck

In [60]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_shank2_nozfilter = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_immuno_nozclean), matching_threshold)

Computing matching costs...
260 potential matches found
TP:228
FP:12
FN:19
Precision:0.95
Recall:0.9230769230769231
F1 score:0.9363449691991786


In [61]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_shank2_init = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_immuno_after_doublecheck), matching_threshold)

Computing matching costs...
255 potential matches found
TP:223
FP:7
FN:24
Precision:0.9695652173913043
Recall:0.902834008097166
F1 score:0.9350104821802935


In [62]:
tp_ids_init = list(zip(*filtered_matches_shank2_init))[1]
tp_ids_nozfilter = list(zip(*filtered_matches_shank2_nozfilter))[1]
s = set(tp_ids_init)
potential_tps = [x for x in tp_ids_nozfilter if x not in s]

In [63]:
inv_filtered_matches_shank2_nozfilter = dict((v, k) for k, v in dict(filtered_matches_shank2_nozfilter).items())

In [64]:
tps_to_keep = []
new_postsyn_immuno_detections = []
new_tps=0
for i in potential_tps:
    if locs_shank2_gt[i][0]<=10 or  locs_shank2_gt[i][0]>=190:
        tps_to_keep.append(i)
        new_tps+=1
        new_postsyn_immuno_detections.append(postsyn_coords_immuno_nozclean[inv_filtered_matches_shank2_nozfilter[i]])
        #here we append the detections matched with these GTs backt o the presyn_coords_immuno list

In [65]:
postsyn_coords_immuno_after_doublecheck_borders = new_postsyn_immuno_detections + postsyn_coords_immuno_after_doublecheck

In [66]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_bassoon_final = compute_molecule_detection_quality(locs_bassoon_gt,np.stack(presyn_coords_immuno_after_doublecheck_borders), matching_threshold)

Computing matching costs...
267 potential matches found
TP:244
FP:20
FN:17
Precision:0.9242424242424242
Recall:0.9348659003831418
F1 score:0.9295238095238095


In [67]:
matching_threshold = 30
precision,recall,fscore,filtered_matches_shank2_final = compute_molecule_detection_quality(locs_shank2_gt,np.stack(postsyn_coords_immuno_after_doublecheck_borders), matching_threshold)

Computing matching costs...
263 potential matches found
TP:231
FP:7
FN:16
Precision:0.9705882352941176
Recall:0.9352226720647774
F1 score:0.9525773195876289


In [68]:
matches_map_doublecheck_final, filtered_matches_immuno_detecion_final = generate_matching_map(presyn_coords_immuno_after_doublecheck_borders,postsyn_coords_immuno_after_doublecheck_borders,matching_threshold=30)

Computing matching costs...
247 potential matches found


In [69]:
#match detected bassoon and shank2 with full gt synapses
full_synapses_gt_and_immuno_after_doublecheck_final = match_detected_and_gt_full_synapses(pre_synapses_gt, post_synapses_gt, full_synapses_gt,filtered_matches_bassoon_final,filtered_matches_shank2_final)

In [70]:
#compute F1 score of full synapse detection
fprecision,frecall,ffscore = compute_full_synapse_detection_quality(full_synapses_gt_and_immuno_after_doublecheck_final,matches_map_doublecheck_final)

FP full-synapse:19
TP full-synapse:219
FN full-synapse:23
Precision:0.9201680672268907
Recall:0.9049586776859504
F1 score:0.9124999999999999
